# Tutorial 06, case 2b: advection diffusion reaction problem with distributed control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega_1 \cup \Omega_2} (y - y_d)^2 dx + \frac{\alpha}{2} \int_{\Omega} u^2 dx$$
s.t.
$$\begin{cases}
- \epsilon \Delta y + \beta \cdot \nabla y + \sigma y = f + u     & \text{in } \Omega\\
                                                    y = g         & \text{on } \partial\Omega
\end{cases}$$

where
$$\begin{align*}
& \Omega                     & \text{domain}\\
& u \in L^2(\Omega)          & \text{control variable}\\
& y \in H^1_0(\Omega)        & \text{state variable}\\
& \alpha > 0                 & \text{penalization parameter}\\
& y_d                        & \text{desired state}\\
& \epsilon > 0               & \text{diffusion coefficient}\\
& \beta \in \mathbb{R}^2     & \text{advection field}\\
& \sigma > 0                 & \text{reaction coefficient}\\
& f                          & \text{forcing term}\\
& g                          & \text{non homogeneous piecewise constant Dirichlet BC}\\
\end{align*}$$
using an adjoint formulation solved by a one shot approach.

The test case is from section 5.2 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import as_vector, grad, inner, Measure, replace, SpatialCoordinate, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace
from dolfinx.fem import locate_dofs_topological, set_bc
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                               assemble_vector, assemble_vector_block, BlockVecSubVectorWrapper,
                               create_vector_block)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/graetz_1.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Function spaces

In [ ]:
Y = FunctionSpace(mesh, ("Lagrange", 1))
U = FunctionSpace(mesh, ("Lagrange", 1))
Q = Y.clone()

### Trial and test functions

In [ ]:
(y, u, p) = (TrialFunction(Y), TrialFunction(U), TrialFunction(Q))
(z, v, q) = (TestFunction(Y), TestFunction(U), TestFunction(Q))

 ### Problem data

In [ ]:
alpha = 0.01
y_d_1 = 0.6
y_d_2 = 1.8
epsilon = 1. / 15.
x = SpatialCoordinate(mesh)
beta = as_vector((x[1] * (1 - x[1]), 0))
sigma = Constant(mesh, 0.)
ff = Constant(mesh, 0.)


def bc_generator(val):
    bc = Function(Y)
    bc.interpolate(lambda x: val * np.ones(x.shape[1]))
    return bc

### Optimality conditions

In [ ]:
state_operator = (epsilon * inner(grad(y), grad(q)) * dx + inner(beta, grad(y)) * q * dx
                  + sigma * y * q * dx)
adjoint_operator = (epsilon * inner(grad(p), grad(z)) * dx - inner(beta, grad(p)) * z * dx
                    + sigma * p * z * dx)
a = [[y * z * (dx(1) + dx(2)), None, adjoint_operator],
     [None, alpha * u * v * dx, - p * v * dx],
     [state_operator, - u * q * dx, None]]
f = [y_d_1 * z * dx(1) + y_d_2 * z * dx(2),
     None,
     ff * q * dx]
a[0][0] += Constant(mesh, 0.) * y * z * dx
a[2][2] = Constant(mesh, 0.) * p * q * dx
f[1] = Constant(mesh, 0.) * v * dx


def bdofs_Y(idx):
    return locate_dofs_topological((Y, Y), mesh.topology.dim - 1, boundaries.indices[boundaries.values == idx])


def bdofs_Q(idx):
    return locate_dofs_topological((Q, Y), mesh.topology.dim - 1, boundaries.indices[boundaries.values == idx])


bc_state = [DirichletBC(bc_generator(idx), bdofs_Y(idx), Y) for idx in (1, 2)]
bc_adjoint = [DirichletBC(bc_generator(0.), bdofs_Q(idx), Q) for idx in (1, 2)]

### Solution

In [ ]:
(y, u, p) = (Function(Y), Function(U), Function(Q))

### Cost functional

In [ ]:
J = (0.5 * inner(y - y_d_1, y - y_d_1) * dx(1) + 0.5 * inner(y - y_d_2, y - y_d_2) * dx(2)
     + 0.5 * alpha * inner(u, u) * dx)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = replace(a[2][0], {q: z})
f_state = replace(f[2], {q: z})

In [ ]:
# Assemble the linear system for the state
A_state = assemble_matrix(a_state, bcs=bc_state)
A_state.assemble()
F_state = assemble_vector(f_state)
apply_lifting(F_state, [a_state], [bc_state])
F_state.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(F_state, bc_state)

In [ ]:
# Solve
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, y.vector)
y.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
J_uncontrolled = mesh.comm.allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.028096831)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_scalar_field_plot(mesh, y, "uncontrolled state")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
bc = bc_state + bc_adjoint
A = assemble_matrix_block(a, bcs=bc)
A.assemble()
F = assemble_vector_block(f, a, bcs=bc)

In [ ]:
# Solve
yup = create_vector_block(f)
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, yup)
yup.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(yup, [Y.dofmap, U.dofmap, Q.dofmap]) as yup_wrapper:
    for yup_wrapper_local, component in zip(yup_wrapper, (y, u, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = yup_wrapper_local

In [ ]:
J_controlled = mesh.comm.allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.001775262)

In [ ]:
pyvista_scalar_field_plot(mesh, y, "state")

In [ ]:
pyvista_scalar_field_plot(mesh, u, "control")

In [ ]:
pyvista_scalar_field_plot(mesh, p, "adjoint")